<div style="text-align:center;">
    <h1 style="font-size:30px;">PROJEKTNI ZADATAK</h1>
    <p style="font-size:25px;">--Prvi dio--</p>
</div>

<p style="font-size:16px;">U ovom dijelu projektnog zadatka kreirana je i istrenirana neuronska mreža nad RCV1 skupom podataka. 
Za rad sa neuronskom mrežom koristila sam biblioteku TensorFlow. Ukoliko se koristi GoogleColab ova biblioteka je automatski instalirana, u slučaju da nije instalirana na računaru to je moguće uraditi iz komandne linije naredbom:</p>

<div style="text-align:center; color:green; font-size:16px;">
    pip install tensorflow
</div>

<p style="font-size:16px;">Za izbor ove biblioteke sam se odlučila zato što je pogodnija za početnike i nudi neposrednu podršku za sparse matrice, dok je u PyTorch-u rad sa sparse matricama nešto manje razvijen. S obzirom da su podaci dati u ovom formatu izabrala sam TensorFlow.</p>

<p style="font-size:16px;">Prvo što radimo je importovanje traženog skupa podataka i dijelimo ga tako da radimo sa 50% uzoraka što je u našem konkretnom slučaju dozvoljeno jer se radi o veoma velikom dataset-u, a inače za bolje performanse preporučuje se naravno treniranje cijelog skupa.</p>

In [1]:
from sklearn.datasets import fetch_rcv1 #importovanje trazenog skupa podataka
from sklearn.model_selection import train_test_split
import numpy as np 

#preuzimanje RCV1 podataka
rcv1 = fetch_rcv1()

#uzimamo podatke i ciljeve iz skupa podataka
X = rcv1.data
y = rcv1.target

In [2]:
#postavljanje random seed za reprodukciju rezultata
np.random.seed(42)

In [3]:
#izracunavanje broja uzoraka koji cini 50% dataset-a
n_samples = X.shape[0]
n_subset = n_samples // 2

In [4]:
#nasumicno biranje 50% uzoraka
indices = np.random.choice(n_samples, n_subset, replace=False)
X_subset = X[indices]
y_subset = y[indices]

<p style="font-size:18px;">U narednih nekoliko ćelija služi analiziramo podatake da bi znali sa čime radimo i na ispravan način osmilili arhitekturu neuronske mreže koju ćemo koristiti.</p>

<p style="font-size:16px;">Ulazni podaci (X_subset) imaju oblik sparse matrice, gde je svaka instanca reprezentovana sa odredjenim brojem karakteristika to jest kolona. Upravo taj broj kolona u matrici X_subset predstavlja broj karakteristika za svaku instancu, što direktno odredjuje broj ulaznih neurona. Dakle, svaki neuron u ulaznom sloju mreže odgovara jednoj karakteristici u podacima i zato zaključujemo da su nam potrebna 47236 neurona u ulaznom sloju.</p>

In [106]:
num_input_neurons = X_subset.shape[1]
print(f"Broj ulaznih neurona: {num_input_neurons}")

Broj ulaznih neurona: 47236


<p style="font-size:16px;">Izlazni podaci (y_subset) su takođe u obliku sparse matrice, gde broj kolona predstavlja broj klasa u slučaju klasifikacije. Broj izlaznih neurona određuje se prema broju različitih izlaznih vrednosti koje mreža treba da predvidi. U slučaju klasifikacije sa više klasa, broj izlaznih neurona je jednak broju klasa, te zaključujemo da će u našem slučaju broj izlaznih neurona biti 103.</p>

In [107]:
num_output_neurons = y_subset.shape[1]
print(f"Broj izlaznih neurona {num_output_neurons}")

Broj izlaznih neurona 103


In [108]:
#nazivi klasa predstavljaju skracenice u odnosu na tematiku clanka
print(f"Nazivi klasa:")
print(rcv1.target_names)

Nazivi klasa:
['C11' 'C12' 'C13' 'C14' 'C15' 'C151' 'C1511' 'C152' 'C16' 'C17' 'C171'
 'C172' 'C173' 'C174' 'C18' 'C181' 'C182' 'C183' 'C21' 'C22' 'C23' 'C24'
 'C31' 'C311' 'C312' 'C313' 'C32' 'C33' 'C331' 'C34' 'C41' 'C411' 'C42'
 'CCAT' 'E11' 'E12' 'E121' 'E13' 'E131' 'E132' 'E14' 'E141' 'E142' 'E143'
 'E21' 'E211' 'E212' 'E31' 'E311' 'E312' 'E313' 'E41' 'E411' 'E51' 'E511'
 'E512' 'E513' 'E61' 'E71' 'ECAT' 'G15' 'G151' 'G152' 'G153' 'G154' 'G155'
 'G156' 'G157' 'G158' 'G159' 'GCAT' 'GCRIM' 'GDEF' 'GDIP' 'GDIS' 'GENT'
 'GENV' 'GFAS' 'GHEA' 'GJOB' 'GMIL' 'GOBIT' 'GODD' 'GPOL' 'GPRO' 'GREL'
 'GSCI' 'GSPO' 'GTOUR' 'GVIO' 'GVOTE' 'GWEA' 'GWELF' 'M11' 'M12' 'M13'
 'M131' 'M132' 'M14' 'M141' 'M142' 'M143' 'MCAT']


<p style="font-size:16px;">Trening i test skup sam podijelila tako da 20% podataka bude u testnom skupu, a preostalih 80% u trening skupu. Uobičajena je praksa da testni skup bude izmedju 20% i 30% jer je tako omogućeno sasvim dovoljno podataka za testiranje modela, dok većina ostaje za obuku što je i ključno za učenje našeg modela.</p>

<p style="font-size:16px;">Shuffle služi za miješanje podataka kako bi se osiguralo da oba skupa budu reprezentativni i da ne postoji pristrasnost u redosledu podataka, dok postavljenjem random_state omogućavamo da svaki put kada se kod pokrene dobijemo isti rezultat, pri tome ne mora biti 42 može i bilo koji drugi broj.</p>

<p style="font-size:16px;">Na kraju za validacioni skup odabrala sam da se koristi pola od testnog skupa, što je 10% od ukupnih podataka. Time je dovoljno podataka (10%) izdvojeno za validaciju modela tokom treniranja, čime se pomaže u odabiru hiperparametara i izbegavanju overfittinga, kao i konačnih 10% podataka bude za završnu evaluaciju performansi modela na potpuno novim i neviđenim podacima.</p>

In [9]:
#podjela 50% uzoraka na trening i test skupove
X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size=0.2,shuffle=True, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [110]:
#samo trenutno za provjeru
print(f'Ukupan broj uzoraka u dataset-u: {n_samples}')
print(f'Broj uzoraka u podskupu (50%): {n_subset}')
print(f'Broj uzoraka u trening skupu: {X_train.shape[0]}')
print(f'Broj uzoraka u test skupu: {X_test.shape[0]}')

Ukupan broj uzoraka u dataset-u: 804414
Broj uzoraka u podskupu (50%): 402207
Broj uzoraka u trening skupu: 321765
Broj uzoraka u test skupu: 40221


In [111]:
n = X_train.shape[1]
print(f" X_train: {n}")
m = X_test.shape[1]
print(f" X_test: {m}")

 X_train: 47236
 X_test: 47236


In [112]:
n = y_train.shape[1]
print(f" y_train: {n}")
m = y_train.shape[1]
print(f" y_train: {m}")

 y_train: 103
 y_train: 103


In [113]:
print(f"Dimenzije:")
print(f"X_train: {X_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"X_test: {X_test.shape}")

Dimenzije:
X_train: (321765, 47236)
X_val: (40221, 47236)
X_test: (40221, 47236)


In [114]:
print(type(X_train))

<class 'scipy.sparse._csr.csr_matrix'>


In [115]:
print(f"Dimenzije:")
print(f"y_train: {y_train.shape}")
print(f"y_val: {y_val.shape}")
print(f"y_test: {y_test.shape}")

Dimenzije:
y_train: (321765, 103)
y_val: (40221, 103)
y_test: (40221, 103)


Postoji nekoliko razloga za narednu konverziju:
 - modeli za klasifikaciju obično očekuju oznake u obliku kategorijskih brojeva umesto one-hot kodovanih vektora
 - funkcija gubitka koju koristim očekuje oznake u obliku kategorija

Tako da ovim korakom vršim pripremu oznaka za dalje treniranje i evaluaciju modela.

In [122]:
#pretvaranje sparse matrica u dense format, a zatim iz one-hot vektora u njihove odgovarajuce kategorije
y_train = y_train.toarray().argmax(axis=1)
y_val = y_val.toarray().argmax(axis=1)
y_test = y_test.toarray().argmax(axis=1)

 - Arhitektura neuronske mreže sačinjena je od **ulaznog**, **izlaznog** i **tri skrivena sloja**. Ranije je pojašnjen razlog za izbor broja ulaznih i izlaznih neurona, a što se tiče skrivenih slojeva u prvoj sloju je veći broj neurona što omogućava mreži da uči složenije obrasce iz podataka, taj broj kasnije sa slojevima smanjujemo radi dodatnog filtriranja informacija.

 - Koristim keras koji je API visokog nivoa za duboko učenje koji omogućava jednostavno kreiranje i treniranje neuronskih mreža. **Sequental** klasa služi za definisanje modela tako što pomoću nje doslovno slažemo slojeve jedan po jedan.

 - Kao aktivaciona funkcija izabrana je **ReLu** jer pomaže u dodavanju nelinearnosti, što je važno za učenje složenih funkcija.

 - U izlaznom sloju aktivaciona funkcija je **Softmax** jer se radi o višeklasnoj klasifikaciji pa je korisno što omogućava modelu da predvidja vjerovatnoće pripadnosti za svaku klasu.

 - Takodje korišćen je i **Dropout**. To je tehnika regularizacije koja pomaže u sprečavanju overfitting-a tako što nasumično isključuje odredjeni procenat neurona tokom treninga. Stopa od 0.5 se često koristi radi balansa izmedju regularizacije i gubitka informacija. Što zapravo znači da će 50% neurona biti isključeno u datom sloju tokom svake iteracije treniranja.

In [123]:
import tensorflow as tf
from tensorflow.keras import layers, models

#definisanje arhitekture neuronske mreze
model = models.Sequential([
    layers.Input(shape=(X_train.shape[1], )),
    layers.Dense(2048, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(103, activation='softmax')
])

U ovom dijelu koda definišemo ključne aspekte načina na koji će model biti treniran.

- Optimizator je algoritam koji model koristi za ažuriranje težina tokom treniranja, tako da se minimizuje funkcija gubitka. **Adam optimizator** je odabran jer često postiže bolje rezultate u kraćem vremenskom roku i nisu potrebna neka naročita podešavanja stope učenja, to jest jednostavniji je za rad. On aptomatski prilagodjava stopu učenja za svaku težinu u modelu na osnovu trenutnih i prošlih gradijenata.

- Funkcija greške **SparseCategoricalCrossentropy** se koristi za klasifikaciju sa integer kodovanim oznakama, umjesto one-hot kodovanja(zato ona prethodna konverzija), ovaj način sam izabrala prvenstveno jer je efikasniji u pogledu memorije, jer je broj klasa i podataka obiman. Funkcija gubitka inače služi da mjeri koliko su predikcije modela tačne u odnosu na stvarne vrijednosti, npr. ako model ne daje visoku vjerovatnoću stvarnoj klasi biće povećana funkcija gubitka.

- **Accuracy** metrika je standardna za klasifikaciju i omogućava nam da pratimo performanse modela tokom obuke.

- **ModelCheckpoint** omogućava povratak na najbolje performanse modela koje odredjuje na osnovu tačnosti validacionog skupa.

In [124]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import ModelCheckpoint

#optimizator, funkcija greske i metrike
model.compile(optimizer=Adam(),
              loss=SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

#da sacuvamo  najbolji model
checkpoint = ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

In [125]:
#radi provjere da li je neuronska mreza kreirana
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_16 (Dense)                │ (None, 2048)           │    96,741,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_12 (Dropout)            │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 1024)           │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 103)            │        52,839 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 99,417,191 (379.25 MB)

 Trainable params: 99,417,191 (379.25 MB)

 Non-trainable params: 0 (0.00 B)

- Trening traje 5 epoha prvenstveno zbog resurnih ograničenja, a manji broj epoha je generalno bolji ako duže obučavanje vodi do overfitting-a tada bi dodatno korstila i early stopping najvjerovatnije

- Veličina batch-a je 64 (model ažurira svoje težine nakon što prodje kroz 64 primjera iz X_train odjednom) što je neka standardna vrijednost koja se uzima koja omogućava balans izmedju brze obuke i stabilnijih gradijenata

- validation_data koristimo za provjeru performansi modela 

- Klasifikacioni izveštaj daje sveobuhvatan pregled performansi modela za svaku klasu, što je korisno za analizu kako se model ponaša prema različitim klasama.

- callbacks osigurava da se najbolji model sacuva tokom treniranja

In [126]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

#treniranje
history = model.fit(X_train, y_train,
                    epochs=5,
                    batch_size=64,
                    validation_data=(X_val, y_val),
                    callbacks=[checkpoint])

#ucitamo informacije koje smo sacuvali o najboljem modelu
best_model = tf.keras.models.load_model('best_model.keras')

#za taj model izvrsimo evaluaciju performansi
loss, accuracy = best_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy}")

#predikcije na testnom skupu
y_pred = best_model.predict(X_test).argmax(axis=1)

#izracunamo metrike
precision = precision_score(y_test, y_pred, average='weighted', zero_division=1)
recall = recall_score(y_test, y_pred, average='weighted', zero_division=1)
f1 = f1_score(y_test, y_pred, average='weighted', zero_division=1)

print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")
print(f"Test F1 Score: {f1}")

print(classification_report(y_test, y_pred, zero_division=1))

Epoch 1/5
5028/5028 ━━━━━━━━━━━━━━━━━━━━ 0s 582ms/step - accuracy: 0.7414 - loss: 0.9630
Epoch 1: val_accuracy improved from -inf to 0.83330, saving model to best_model.keras
5028/5028 ━━━━━━━━━━━━━━━━━━━━ 2943s 585ms/step - accuracy: 0.7414 - loss: 0.9629 - val_accuracy: 0.8333 - val_loss: 0.5611
Epoch 2/5
5028/5028 ━━━━━━━━━━━━━━━━━━━━ 0s 584ms/step - accuracy: 0.8672 - loss: 0.4562
Epoch 2: val_accuracy improved from 0.83330 to 0.83576, saving model to best_model.keras
5028/5028 ━━━━━━━━━━━━━━━━━━━━ 2949s 587ms/step - accuracy: 0.8672 - loss: 0.4562 - val_accuracy: 0.8358 - val_loss: 0.5511
Epoch 3/5
5028/5028 ━━━━━━━━━━━━━━━━━━━━ 0s 570ms/step - accuracy: 0.9024 - loss: 0.3309
Epoch 3: val_accuracy did not improve from 0.83576
5028/5028 ━━━━━━━━━━━━━━━━━━━━ 2874s 572ms/step - accuracy: 0.9024 - loss: 0.3309 - val_accuracy: 0.8349 - val_loss: 0.5990
Epoch 4/5
5028/5028 ━━━━━━━━━━━━━━━━━━━━ 0s 568ms/step - accuracy: 0.9285 - loss: 0.2459
Epoch 4: val_accuracy did not improve from 0.8

<div style="text-align:center;">
    <p style="font-size:25px;">--Drugi dio--</p>
</div>

U drugom dijelu projektnog zadatka trebalo je da istreniramo LightGBM nad istim skupom podataka samo u ovom slučaju koristiti cijeli dataset, a ne samo 50%, pa sam ga zato opet učitala i podijelila podatke. Mada idealno bi bilo da je korišćen isti raspored podataka za treniranje i neuronske mreže i LightGBM-a jer je cilj bio uporediti performanse zbog restartovanja kernela to nije bilo moguće, pa će i ovi rezultati poslužiti.

<p style="font-size:16px;">Da bismo koristili Optunu za optimizaciju hiperparametara i LightGBM za treniranje modela potrebno je u komandnoj liniji pokrenuti sledeću naredbu. Takodje ako nije instalirana unaprijed i biblioteka sckit-learn i nju instaliramo.</p>

<div style="text-align:center; color:green; font-size:16px;">
    pip install optuna lightgbm scikit-learn
</div>

In [1]:
#importovanje potrebnih biblioteka
import numpy as np
from sklearn.datasets import fetch_rcv1
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import precision_score, recall_score, accuracy_score
import optuna
from scipy.sparse import csr_matrix
import time

In [2]:
# Učitavanje RCV1 dataset-a
rcv1 = fetch_rcv1()
X = rcv1.data  # Sparse matrica (CSR format)
y = rcv1.target  # Sparse matrica (multi-label)

<p style="font-size:22px;">Pojašnjenje konverzija i načina klasifikacije</p>

Orginalna ciljana promjenljiva y u RCV1 dataset-u je u multi-label formatu, gdje jedan uzorak može pripadati više klasa istovremeno. Medjutim, u ovom projektu prije svega zbog jednostavnosti fokusiramo se na problem višeklasifikacije tj. multi-class problem, gdje svaki uzorak treba da pripada samo jednoj klasi. Naravno važno je napomenuti da sa ovim pristupom gubimo neke informacije, što nije idealno, ali je neizbježno zbog načina na koji multi-class funkcioniše što ću pojasniti kasnije.

Prvo pretvaramo y u dense format da bismo mogli pronaći najvažnije oznake (labels). Svaki red predstavlja jedan uzorak, a svaka kolona jednu oznaku.

In [3]:
# Pretvaranje y iz sparse u dense format
y_dense = y.toarray()

In [4]:
# Provjera broja uzoraka u svakoj klasi u originalnom multi-label skupu
class_counts = np.sum(y_dense, axis=0)
print(f"Broj uzoraka u svakoj klasi: {class_counts}")
print(f"Broj klasa sa uzorcima: {np.sum(class_counts > 0)}")


Broj uzoraka u svakoj klasi: [ 24325  11944  37410   7410 151785  81890  23211  73092   1920  42155
  18313  11487   2636   5871  52817  43374   4671   7406  25403   6119
   2625  32153  40509   4299   6648   1115   2084  15332   1210   4835
  11355  10272  11878 381327   8568  27100   2182   6603   5659    939
   2177    376    200   1206  43130  15768  27405   2415   1701     52
    111  17035   2136  21280   2933  12634   2290    391   5268 119920
  20672   3307   2107   2360   8404   2124    260   2036   4300     40
 239267  32219   8842  37739   8657   3801   6261    313   6030  17241
      5    844   2802  56878   5498   2849   2410  35317    680  32615
  11532   3878   1869  48696  26036  53634  28185  26752  85440  47708
  12130  21957 204820]
Broj klasa sa uzorcima: 103


U multi-label okruženju, jedan uzorak može imati više oznaka, ali za problem višeklasifikacije, svaki uzorak mora pripadati samo jednoj klasi. Funkcija np.argmax se koristi za izbor oznake sa najvećom vrednošću (ili značajem) za svaki uzorak. Ovo efikasno smanjuje problem sa više oznaka na problem sa jednom oznakom (multi-class) tako što se bira najistaknutija oznaka.
- np.argmax je funkcija koja vraća indeks najveće vrijednosti duž određene ose (axis) u nizu
- axis odredjuje duž koje ose treba da se izvrši operacija, u ovom slučaju axis=1 znači da se operacija izvršava duž redova

In [5]:
# Pretvaranje multi-label u multi-class (biramo najvažniju labelu)
y_multi_class = np.argmax(y_dense, axis=1)

In [6]:
# Provjera broja klasa prije i poslije konverzije
num_classes_original = np.unique(y_multi_class).shape[0]
print(f"Broj klasa u originalnom y: {num_classes_original}")

Broj klasa u originalnom y: 37


In [7]:
# Podjela na train, val i test
X_train, X_temp, y_train, y_temp = train_test_split(X, y_multi_class, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [8]:
print("Dimenzije y_train:", y_train.shape)
print("Dimenzije y_val:", y_val.shape)
print("Dimenzije y_test:", y_test.shape)


Dimenzije y_train: (563089,)
Dimenzije y_val: (120662,)
Dimenzije y_test: (120663,)


**param{...}** - u ovom dijelu se definišu hiperparametri modela koje Optuna treba da optimizuje. Svaki parametar ima odredjeni opseg ili skup mogućih vrijednosti iz kojeg Optuna nasumično bira i testira različite kombinacije.

- **objective** definiše da se model trenira za multi-class klasifikaciju
- **multi_logloss** mjeri koliko su predikcije modela loše

Objašnjenje izbora opsega hiperparametara:
- **learning_rate** odredjuje koliko će model korigovati svoje greške u svakom iterativnnom koraku; manja vrijednost znači sporije i stabilnije učenje, a veća može ubrzati učenje, ali postoji rizik da model preskoči optimalno rješenje(grafik sa vježbi)
- **num_leaves** definiše maksimalan broj listova u svakom stablu; više listova omogućava modelu da napravi složenije odluke, ali može dovesti do overfitting-a
- **feature_fraction** kontroliše dio karakteristika koje će se koristiti u svakoj boosting iteraciji; vrijednosti bliže 1.0 znače da će se skoro sve karakteristike koristiti, ako izaberemo manje sprječavamo mogućnost overfitting-a
- **bagging_fraq** definiše koliko često će se bagging koristiti u iteracijama
- **min_child_samples** definiše minimalan broj uzoraka koji je potreban da bi se formirao list u stablu; veća vrijednost manja šansa za overfitting 

Biramo opsege koji su dovoljno široki da bi model mogao da testira različite strategije učenja sa različitim karakteristikama, a da opet ne bi došlo do overfitting-a i naravno obraćamo pažnju da ne pretjeramo sa opsegom jer bi to moglo dosta vremenski da traje npr. za veoma mali lr učenje može dosta da se oduži.

In [9]:
# Definisanje funkcije cilja za Optuna
def objective(trial):
    start_time = time.time() # zapocnemo mjerenje za svaki trial
    print("Pokretanje novog triala ...")
    param = {
        "objective": "multiclass",
        "metric": "multi_logloss",
        "num_class": num_classes_original,
        "verbosity": -1,
        "boosting_type": "gbdt",
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1),
        "num_leaves": trial.suggest_int("num_leaves", 16, 128),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
        #"bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),    
    }
    
    # Treniranje modela
    print(f"Treniranje modela sa parametrima: {param}")
    model = lgb.LGBMClassifier(**param)
    model.fit(X_train, y_train)
    
    
    # Predvidjanje i evaluacija
    print("Predviđanje na validacionom skupu...")
    y_pred = model.predict(X_val)
    precision = precision_score(y_val, y_pred, average='weighted')  # Preciznost za multi-class problem

    # Mjerenje i ispis vremena izvršavanja
    end_time = time.time()
    trial_time = end_time - start_time
    print(f"Trial {trial.number} završen za {trial_time:.2f} sekundi s preciznošću: {precision:.4f}")
    
    # Vraca se preciznost modela, koja ce se koristiti za procjenu kvaliteta hiperparametara u toj iteraciji
    return precision

Kreiramo navu Optuna studiju koja predstavlja eksperiment u kojem Optuna traži najbolju kombinaciju hiperparametara.
 - **maximize** opcija govori Optuni da maksimizira neku metriku, u našem slučaju tačnost.
 - Zatim se pokreće proces optimizacije gdje funkciji šaljemo **objective** funkciju koju smo ranije definisali i ona govori kako da se model ternira i evaluira 
  - i **n_trials** govori koliko pokušaja optimizacije da se izvrši, naravno bolje je staviti više pokušaja, ali zbog vremenske složenosti ja sam odabrala 5 za početak, što se nije ispostavilo kao loše jer je i dalje tačnost dosta visoka

In [10]:
# Optuna za hiperparametarsku pretragu
print("Pokretanje Optuna studije...")
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

print("Broj završenih trial-a: {}".format(len(study.trials)))
print("Najbolji trial:")
trial = study.best_trial
print("  Vrijednost: {}".format(trial.value))
print("  Parametri: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2024-08-23 21:03:29,669] A new study created in memory with name: no-name-0c7182dd-982a-45b1-b2ac-dbbbcef9af65


Pokretanje Optuna studije...
Pokretanje novog triala ...
Treniranje modela sa parametrima: {'objective': 'multiclass', 'metric': 'multi_logloss', 'num_class': 37, 'verbosity': -1, 'boosting_type': 'gbdt', 'learning_rate': 0.022572101504552654, 'num_leaves': 109, 'feature_fraction': 0.8271740338049249, 'bagging_freq': 4, 'min_child_samples': 39}
Predviđanje na validacionom skupu...


[I 2024-08-23 22:33:57,343] Trial 0 finished with value: 0.8198549173252715 and parameters: {'learning_rate': 0.022572101504552654, 'num_leaves': 109, 'feature_fraction': 0.8271740338049249, 'bagging_freq': 4, 'min_child_samples': 39}. Best is trial 0 with value: 0.8198549173252715.


Trial 0 završen za 5427.65 sekundi s preciznošću: 0.8199
Pokretanje novog triala ...
Treniranje modela sa parametrima: {'objective': 'multiclass', 'metric': 'multi_logloss', 'num_class': 37, 'verbosity': -1, 'boosting_type': 'gbdt', 'learning_rate': 0.060323122675371856, 'num_leaves': 75, 'feature_fraction': 0.9461263471443339, 'bagging_freq': 7, 'min_child_samples': 45}
Predviđanje na validacionom skupu...


[I 2024-08-23 23:47:53,996] Trial 1 finished with value: 0.7700943690373268 and parameters: {'learning_rate': 0.060323122675371856, 'num_leaves': 75, 'feature_fraction': 0.9461263471443339, 'bagging_freq': 7, 'min_child_samples': 45}. Best is trial 0 with value: 0.8198549173252715.


...

Trial 2 završen za 5378.02 sekundi s preciznošću: 0.7419
Pokretanje novog triala ...
Treniranje modela sa parametrima: {'objective': 'multiclass', 'metric': 'multi_logloss', 'num_class': 37, 'verbosity': -1, 'boosting_type': 'gbdt', 'learning_rate': 0.04033652751323631, 'num_leaves': 84, 'feature_fraction': 0.8604216815765149, 'bagging_freq': 3, 'min_child_samples': 9}
Predviđanje na validacionom skupu...


[I 2024-08-24 03:40:55,891] Trial 3 finished with value: 0.7875761711229381 and parameters: {'learning_rate': 0.04033652751323631, 'num_leaves': 84, 'feature_fraction': 0.8604216815765149, 'bagging_freq': 3, 'min_child_samples': 9}. Best is trial 0 with value: 0.8198549173252715.


Trial 3 završen za 8603.82 sekundi s preciznošću: 0.7876
Pokretanje novog triala ...
Treniranje modela sa parametrima: {'objective': 'multiclass', 'metric': 'multi_logloss', 'num_class': 37, 'verbosity': -1, 'boosting_type': 'gbdt', 'learning_rate': 0.01469002324876744, 'num_leaves': 127, 'feature_fraction': 0.8663276870770615, 'bagging_freq': 7, 'min_child_samples': 29}
Predviđanje na validacionom skupu...


[I 2024-08-24 05:34:16,723] Trial 4 finished with value: 0.8106175677993451 and parameters: {'learning_rate': 0.01469002324876744, 'num_leaves': 127, 'feature_fraction': 0.8663276870770615, 'bagging_freq': 7, 'min_child_samples': 29}. Best is trial 0 with value: 0.8198549173252715.


Trial 4 završen za 6800.81 sekundi s preciznošću: 0.8106
Broj završenih trial-a: 5
Najbolji trial:
  Vrijednost: 0.8198549173252715
  Parametri: 
    learning_rate: 0.022572101504552654
    num_leaves: 109
    feature_fraction: 0.8271740338049249
    bagging_freq: 4
    min_child_samples: 39


In [11]:
# Treniranje konacnog modela sa najboljim hiperparametrima
print("Treniranje konačnog modela sa najboljim parametrima...")
best_params = study.best_params
final_model = lgb.LGBMClassifier(**best_params, objective='multiclass')

# Fitovanje modela na cijelom trening skupu
final_model.fit(X_train, y_train)

# Evaluacija na testnom skupu
print("Evaluacija na testnom skupu...")
y_pred = final_model.predict(X_test)
test_precision = precision_score(y_test, y_pred,average='weighted')
test_recall = recall_score(y_test, y_pred, average='weighted')
test_accuracy = accuracy_score(y_test, y_pred)
print(f"Tačnost na testnom skupu: {test_accuracy}")
print(f"Preciznost na testnom skupu: {test_precision}")
print(f"Preciznost na testnom skupu: {test_recall}") #Ovde je greska treba pisati odziv ne preciznost

Treniranje konačnog modela sa najboljim parametrima...
Evaluacija na testnom skupu...
Tačnost na testnom skupu: 0.8272627068778333
Preciznost na testnom skupu: 0.8194036966397518
Preciznost na testnom skupu: 0.8272627068778333


Napomenta: greška u kucanju zadnji print je odziv ne preciznost.

POREDJENJE

Performanse neuronske mreže i LightGBM-a možemo porediti po sve tri metrike: tačnost, preciznost i odziv. Iako su rezultati približni, neuronska mmreža je malo bolja u odnosu na LightGBM. A i u pogledu brzine izvršavanja pokazala se kao bolji model.